In [40]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from statistics import median
import pickle
import csv
import multiprocess
import warnings
warnings.filterwarnings("ignore")

In [41]:
MAX_BATCH = [1, 2, 4, 8, 16]
algorithm = ['BATCHBISECT', 'BATCH4', 'BATCHSTOP4']
confidence = list(range(2,21,1))

In [42]:
projects = ['vagrant.csv', 'puppet.csv', 'graylog2-server.csv', 'heroku.csv', 'rails.csv', 'gradle.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'rubinius.csv', 'open-build-service.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv']
data_path = '../data/exact_data/'

In [53]:
result_file = open('trial_results.csv', 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'confidence', 'project_reqd_builds', 'project_missed_builds', 'project_saved_builds', 'sbs_delays', 'testall_size', 'total_batch_delay', 'batch_delays', 'ci']
writer = csv.writer(result_file)
writer.writerow(result_headers)
result_file.close()

In [54]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [55]:
def pd_get_train_test_data(file_path, first_failures=True):
    columns = ['version', 'tr_build_id', 'git_num_all_built_commits', 'git_diff_src_churn', 'git_diff_test_churn', 'gh_diff_files_modified', 'tr_status']
    X = pd.read_csv(file_path, usecols = columns)
    X['tr_status'] = output_values(X['tr_status'])
    if first_failures:
        X = get_first_failures(X)
    #X.drop('tr_status', inplace=True, axis=1)
    
    #return X, Y
    return X

In [56]:
batch_total = 0
batch_durations = 0

In [57]:
def batch_bisect(grouped_batch, actual_group_results):
    global batch_total
 
    batch_total += 1
    
    if len(actual_group_results) == 1:
        return
    
    if 0 in actual_group_results:
        half_batch = len(actual_group_results)//2
        batch_bisect(grouped_batch[:half_batch], actual_group_results[:half_batch])
        batch_bisect(grouped_batch[half_batch:], actual_group_results[half_batch:])

In [58]:
def batch_stop_4(grouped_batch, actual_group_results):
    global batch_total
    
    batch_total += 1

    if len(actual_group_results) <= 4:
        if 0 in actual_group_results:
            batch_total += 4
        return
    
    if 0 in actual_group_results:
        half_batch = len(actual_group_results)//2
        batch_stop_4(grouped_batch[:half_batch], actual_group_results[:half_batch])
        batch_stop_4(grouped_batch[half_batch:], actual_group_results[half_batch:])

In [59]:
def get_predictions(p):
    
    test_file = "../data/exact_data/" + p + '.csv'
    project = pd_get_train_test_data(test_file, first_failures=False)
    
    
    #Received all predictions
    predictions = []
    Y_test = []
    X_test = pd.DataFrame()
    
    for ver in range(1, 11):

        model_file_name = '../../RQ2-Models/' + p + '_models/rq2_' + p + '_' + str(ver) + '_best_model.pkl'
        model_file = open(model_file_name, 'rb')
        predictor = pickle.load(model_file)
        threshold = pickle.load(model_file)
        

        x_test = project [ project['version'] == ver]
        if len(x_test) == 0:
            continue
        y_test = x_test['tr_status'].tolist()

        x_test.drop('tr_build_id', inplace=True, axis=1)
        x_test.drop('tr_status', inplace=True, axis=1)
        x_test.drop('version', inplace=True, axis=1)
        
        preds = predictor.predict(x_test)
        
        X_test = X_test.append(x_test)
        predictions.extend(preds)
        Y_test.extend(y_test)
    
    return predictions, Y_test, X_test

In [60]:
def static_rule(p):
    
    
    global batch_total
    global batch_durations
    
    p = p.split('.')[0]
    
    #result_file_name = p + '_rq2_results.csv'
    result_file = open('trial_results.csv', 'a+')
    writer = csv.writer(result_file)
    
    commit, Y_test, X_test = get_predictions(p)
    
    for alg in algorithm:
        for max_batch_size in MAX_BATCH:
                        
            if alg == 'BATCH4':
                if max_batch_size != 4:
                    continue
            
            if alg == 'BATCHSTOP4':
                if max_batch_size < 4:
                    continue
                    
            print('Processing {} at batch size {} for {}'.format(alg, max_batch_size, p))


            Y_result = []
            grouped_batch = []
            actual_group_results = []
            group_duration = []
            num_feature = 4 
            length_of_test = len(Y_test)

            project_reqd_builds = []
            project_missed_builds = []
            project_build_duration = []
            project_saved_builds = []
            project_delays = []
            project_bad_builds = []
            project_batch_delays = []
            project_batch_medians = []
            project_ci = []

            print('Processing {}'.format(p))
            for c in confidence:
                ci = [Y_test[0]]
                batch_median = []
                batch_delays = 0

                pass_streak = Y_test[0]
                total_builds = 0
                missed_builds = 0
                miss_indexes = []
                build_indexes = []
                delay_durations = []

                if pass_streak == 0:
                    saved_builds = 0
                else:
                    saved_builds = 1

                index = 1

                while index < len(X_test):
                    value = commit[index]
                    #we're setting a confidence of 'c' builds on SBS, if more than 'c' passes have been suggested in a row, we don't want to trust sbs
                    
                    #if predict[0][1] > threshold:
                    #    value = 1
                    #else:
                    #    value = 0
                    #print('Build {} : predict_proba={}\tprediction={}'.format(index, predict, value))
                    
                    
                    if pass_streak < c :
                        
                        if value == 0:
                            while True:

                                grouped_batch = list(X_test[index : index+max_batch_size])
                                actual_group_results = list(Y_test[index : index+max_batch_size])

                #                 if len(grouped_batch) < max_batch_size:
                #                     grouped_batch.append(index)
                #                     actual_group_results.append(Y_test[index])
                #                     group_duration.append(Y_duration[index])
                                if alg == 'BATCH4':
                                    if len(actual_group_results) != max_batch_size:
                                        fb = 0
                                        while fb < len(actual_group_results):
                                            #miss_indexes.append(index)
                                            batch_delays += len(actual_group_results) - fb
                                            batch_median.append(max_batch_size-fb-1)
                                            ci.append(0)
                                            fb += 1
                                            index += 1
                                            total_builds += 1
                                    else:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                        ci.extend([0 for clb in range(max_batch_size)])
                                        total_builds += 1
                                        

                                        if 0 in actual_group_results:
                                            total_builds += max_batch_size
                                            
                                        #grouped_batch.clear()
                                        #actual_group_results.clear()
                                        #group_duration.clear()

                                elif alg == 'BATCHBISECT':
                                    if len(actual_group_results) != max_batch_size:
                                        fb = 0
                                        while fb < len(actual_group_results):
                                            total_builds += 1
                                            ci.append(0)
                                            batch_delays += len(actual_group_results) - fb
                                            batch_median.append(max_batch_size-fb-1)
                                            fb += 1
                                            index += 1
                                    else:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_total = 0
                                        
                                        batch_bisect(grouped_batch, actual_group_results)
                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        ci.extend([0 for clb in range(max_batch_size)])
                                        batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                        total_builds += batch_total

                                        #grouped_batch.clear()
                                        #actual_group_results.clear()
                                elif alg == 'BATCHSTOP4':
                                    if len(actual_group_results) != max_batch_size:
                                        fb = 0
                                        while fb < len(actual_group_results):
                                            total_builds += 1
                                            ci.append(0)
                                            batch_delays += len(actual_group_results) - fb
                                            batch_median.append(max_batch_size-fb-1)
                                            fb += 1
                                            index += 1
                                    else:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_total = 0
                                        batch_durations = 0

                                        batch_stop_4(grouped_batch, actual_group_results)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        ci.extend([0 for clb in range(max_batch_size)])
                                        batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                        total_builds += batch_total


                                        #grouped_batch.clear()
                                        #actual_group_results.clear()
                                        #group_duration.clear()
                                elif alg == 'NOBATCH':
                                    if len(grouped_batch) != max_batch_size:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        total_builds += 1
                                        total_duration += max(group_duration)

                                        #grouped_batch.clear()
                                        #actual_group_results.clear()
                                        #group_duration.clear()


                                if 0 in actual_group_results:
                                    index += max_batch_size
                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    
                                else:
                                    break
                            index += max_batch_size
                            pass_streak = 1
                            grouped_batch.clear()
                            actual_group_results.clear()
                            
                                
#                             total_builds += 1
#                             total_duration += Y_duration[index]
#                             if len(miss_indexes) > 0:
#                                 if miss_indexes[-1] < index:
#                                     for l in range(len(miss_indexes)):
#                                         e = miss_indexes.pop()
#                                         delay_durations.append(index - e + 1)
                        else:
                            pass_streak += 1
                            ci.append(1)
                            saved_builds += 1
                            if Y_test[index] == 0:
                                missed_builds += 1
                                miss_indexes.append(index)

                            #seeing only one build
                            index += 1

                    else:
                        while True:

                            grouped_batch = list(X_test[index : index+max_batch_size])
                            actual_group_results = list(Y_test[index : index+max_batch_size])
                            

                            #print(grouped_batch)
                            #print(actual_group_results)

            #                 if len(grouped_batch) < max_batch_size:
            #                     grouped_batch.append(index)
            #                     actual_group_results.append(Y_test[index])
            #                     group_duration.append(Y_duration[index])

                            if alg == 'BATCH4':
                                if len(actual_group_results) != max_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(max_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)
                                    
                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    ci.extend([0 for clb in range(max_batch_size)])
                                    batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                    total_builds += 1
                                    

                                    if 0 in actual_group_results:
                                        total_builds += max_batch_size
                                        

                                    #grouped_batch.clear()
                                    #actual_group_results.clear()
                                    #group_duration.clear()

                            elif alg == 'BATCHBISECT':
                                if len(actual_group_results) != max_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(max_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:

                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = 0
                                    

                                    batch_bisect(grouped_batch, actual_group_results)

                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                    
                                    ci.extend([0 for clb in range(max_batch_size)])
                                    total_builds += batch_total
                                    

                                    #print(actual_group_results)
                                    #print('Total builds = {}, batch_total = {}, batch_delays={}'.format(total_builds, batch_total, batch_delays))
                                    #print('Total builds = {}, batch_delays={}'.format(total_builds, batch_delays))

                                    #grouped_batch.clear()
                                    #actual_group_results.clear()
                                    #group_duration.clear()
                            elif alg == 'BATCHSTOP4':
                                if len(actual_group_results) != max_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(max_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:

                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = 0
                                    

                                    batch_stop_4(grouped_batch, actual_group_results)

                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    batch_median.extend([max_batch_size-clb-1 for clb in range(max_batch_size)])
                                    ci.extend([0 for clb in range(max_batch_size)])
                                    total_builds += batch_total
                                    

                                    #print(actual_group_results)
                                    #print('Total builds = {}, batch_total = {}, batch_delays={}'.format(total_builds, batch_total, batch_delays))

                                    #grouped_batch.clear()
                                    #actual_group_results.clear()
                                    #group_duration.clear()
                            elif alg == 'NOBATCH':
                                if len(grouped_batch) == max_batch_size:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)
                                    
                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    total_builds += 1
                                    total_duration += max(group_duration)

                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    group_duration.clear()

                            
                            if 0 in actual_group_results:
                                index += max_batch_size
                                grouped_batch.clear()
                                actual_group_results.clear()
                                
                            else:
                                break
                        index += max_batch_size
                        pass_streak = 1
                        grouped_batch.clear()
                        actual_group_results.clear()
                        
                mi = 0
                while len(miss_indexes) > 0:
                        m_index = miss_indexes.pop()
                        delay_durations.append(length_of_test - m_index + 1)

                project_reqd_builds.append(total_builds)
                project_missed_builds.append(missed_builds)
                project_saved_builds.append(saved_builds)
                project_delays.append(delay_durations)
                project_batch_delays.append(batch_delays)
                project_batch_medians.append(batch_median)
                project_ci.append(ci)
                
                if len(ci) != len(commit):
                    print(len(ci))
                    print(len(commit))
                    print('PROBLEM!')
                else:
                    print('NO PROBLEM!')

            #print(p)
            #print(project_reqd_builds)
            #print(project_missed_builds)
            #print(project_build_duration)
            #print(project_saved_builds)
            #print(project_delays)
            #print(project_batch_delays)
            
            for i in range(len(confidence)):
                #print([p, alg, max_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, project_build_duration[i], 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i]])
                writer.writerow([p, alg, max_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i], project_batch_medians[i], project_ci[i]])
    result_file.close()

In [61]:
for pr in projects:
    static_rule(pr)

Processing BATCHBISECT at batch size 1 for vagrant
Processing vagrant
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 2 for vagrant
Processing vagrant
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 4 for vagrant
Processing vagrant
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 8 for vagrant
Processing vagrant
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!


NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 8 for heroku
Processing heroku
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 16 for heroku
Processing heroku
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCH4 at batch size 4 for heroku
Processing heroku
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch s

Processing BATCHBISECT at batch size 16 for jruby
Processing jruby
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCH4 at batch size 4 for jruby
Processing jruby
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 4 for jruby
Processing jruby
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 8 for jruby
Processing jruby
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM

Processing BATCHBISECT at batch size 16 for rubinius
Processing rubinius
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCH4 at batch size 4 for rubinius
Processing rubinius
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 4 for rubinius
Processing rubinius
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 8 for rubinius
Processing rubinius
NO PROBLEM!
NO PROBLEM!
NO PROBLEM

NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 16 for loomio
Processing loomio
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCH4 at batch size 4 for loomio
Processing loomio
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 4 for loomio
Processing loomio
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 8 for loomio
Processing loomio
NO PROBLEM!
NO PROBLEM!
NO

NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 4 for cloud_controller_ng
Processing cloud_controller_ng
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 8 for cloud_controller_ng
Processing cloud_controller_ng
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 16 for cloud_controller_ng
Processing cloud_controller_ng
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!


NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 8 for geoserver
Processing geoserver
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHSTOP4 at batch size 16 for geoserver
Processing geoserver
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
Processing BATCHBISECT at batch size 1 for orbeon-forms
Processing orbeon-forms
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PROBLEM!
NO PR